# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [41]:
import pandas as pd
import numpy as np
from scipy import stats


In [12]:
data = pd.io.stata.read_stata('us_job_market_discrimination.dta')

## Callback data

In [25]:
# number of black-sounding names
b_names = len(data[data.race=='b'])
print('The number of black-sounding names is: {}'.format(b_names))

# number of callbacks for black-sounding names
b_c = sum(data[data.race=='b'].call)
print('The number of callbacks for black-sounding names is: {}'.format(b_c))

# percent callback for black-sounding names
b_p = b_c/b_names
print('The percent callback for black-sounding names is: {:0.5}'.format(b_p * 100))



# number of white-sounding names
w_names = len(data[data.race=='w'])
print('The number of white-sounding names is: {}'.format(w_names))

# number of callbacks for white-sounding names
w_c = sum(data[data.race=='w'].call)
print('The number of callbacks for white-sounding names is: {}'.format(w_c))

# percent callback for white-sounding names
w_p = w_c/w_names
print('The percent callback for white-sounding names is: {:0.5}'.format(w_p * 100))


The number of black-sounding names is: 2435
The number of callbacks for black-sounding names is: 157.0
The percent callback for black-sounding names is: 6.4476
The number of white-sounding names is: 2435
The number of callbacks for white-sounding names is: 235.0
The percent callback for white-sounding names is: 9.6509


In [14]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## Question 1:

- What test is appropriate for this problem?
- Does CLT apply? 

There are two tests that could be appropriate for this problem. The first test, a z-test for proportions can be used since the samples are independent, each sample includes at least 10 success and failures (white vs black sounding names), and each population is at least 20 times as big as its sample (conservatively, this would equate to 48,700 individuals with white sounding names and 48,700 individual with black-sounding names). The other test that could be used on this problem is resampling. 

Does the Central Limit Theorem apply? Requirements for CLT are: the samples must be independent, the data must be sampled randomly, and the sample size must be 'sufficiently large'. In this case, the samples are independent, the data is presumed to have been sampled randomly, with a data set of 2,435 white-sounding names and 2,435 black-sounding names, and the sample size is large enough to be representative of a bigger population set. 


## Question 2:
- What are the null and alternate hypotheses?

**Null hypothesis (Ho):** the probability of success (getting a callback) is the same for resumes with both white-sounding names and black-sounding names:

                        Ho: rate_w - rate_b = 0
    

**Alternative hypothesis (Ha):** the probability of success is not the same for resumes with white-sounding names as it is for those with black-sounding names: 

                         Ha: rate_w - rate_b != 0

In [18]:
w = data[data.race=='w']
b = data[data.race=='b']

## Question 3: 
- Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

### Frequentist Approach

In [20]:
def ztest_proportions_two_samples(r1, n1, r2, n2, one_sided=False):
    """Returns the z-statistic and p-value for a 2-sample Z-test of proportions"""
    p1 = r1/n1
    p2 = r2/n2
    
    p = (r1+r2)/(n1+n2)
    se = np.sqrt(p*(1-p)*(1/n1+1/n2))
    
    z = (p1-p2)/se
    p = 1-stats.norm.cdf(abs(z))
    p *= 2-one_sided
    return z, p

In [31]:
# 95% Confidence Interval
prop_diff = w_p - b_p
print('Observed difference in proportions: \t {}\n'.format(prop_diff))

z_crit = 1.96
p_1 = w_p*(1-w_p)/w_names
p_2 = b_p*(1-b_p)/b_names

ci_high = prop_diff + z_crit*(np.sqrt(p_1 + p_2))
ci_low = prop_diff - z_crit*(np.sqrt(p_1 + p_2))

z_stat, p_val = ztest_proportions_two_samples(w_c, w_names, b_c, b_names)
print('z-stat: \t {:0.5}\np-value: \t {:0.5}'.format(z_stat, p_val))

print('95% confidence interval: \t {:0.5} - {:0.5}'.format(ci_low, ci_high))

# Margin of Error
moe = (ci_high - ci_low)/2
print('Margin of error: +/-{:0.5}'.format(moe))

Observed difference in proportions: 	 0.032032854209445585

z-stat: 	 4.1084
p-value: 	 3.9839e-05
95% confidence interval: 	 0.016777 - 0.047288
Margin of error: +/-0.015255


### Bootstrap approach

In [38]:
# Construct arrays of data: white-sounding names, black-sounding names
all_callbacks = np.array([True] * int(np.sum(data.call)) + [False] * int(len(data) - np.sum(data.call)))

size = 10000

bs_reps_diff = np.empty(size)

for i in range(size):
    w_bs_replicates = np.sum(np.random.choice(all_callbacks, size=w_names))
    b_bs_replicates = np.sum(np.random.choice(all_callbacks, size=b_names))
    
    bs_reps_diff[i] = (w_bs_replicates - b_bs_replicates)/b_names
    
bs_p_value = np.sum(bs_reps_diff >= prop_diff) / len(bs_reps_diff)

bs_ci = np.percentile(bs_reps_diff, [2.5, 97.5])
bs_mean_diff = np.mean(bs_reps_diff)

print('obs diff: {}\n'.format(prop_diff))
print('BOOTSTRAP RESULTS\np-value: {}\n95% conf. int.: {}'.format(bs_p_value, bs_ci))

obs diff: 0.032032854209445585

BOOTSTRAP RESULTS
p-value: 0.0
95% conf. int.: [-0.01519507  0.01519507]


**Analysis:**

The p value for the bootstrap method and the frequentist approach are effectively zero, putting the values below the p=0.05 threshold, so the null hypothesis must be rejected, in favor of the alternative hypothesis. The acceptance of the alternative hypothesis suggests that the perception of race based on the name on a resume does have an effect of whether or not a an applicant will receive a callback. 

## Question 4: 
- Write a story describing the statistical significance in the context or the original problem.

The data consistently and conclusively revealed that there is a difference in callback success for resumes with black-sounding names compared to resumes with white-sounding names. Resumes with black-sounding names receive a callback 6.45% of the time while resumes with white-sounding names receive a callback 9.65% of the time, a nearly 50% increase in the callback rates. 

## Question 5: 
- Does your analysis mean that race/name is the most important factor in callback success? 
- Why or why not? If not, how would you amend your analysis?

Per the study's website (listed above under 'Resources'), the resumes that were sent in the study were identical in every way except for the assignment of white-sounding or black-sounding names. Given this information, the analysis performed would indicate that racially associated names are an important factor in callback success. However, the study also noted that the name bias was not necessarily applicable in all neighborhoods; the study also found that predominantly black communities in Chicago had a significantly lower difference in the rate of callbacks between white-sounding and black-sounding names on resumes. Given these findings, it is possible that other factors may impact callback success more than race/name but it is clear that race/name have an impact on callback success. 